In this notebook we now explore how to use litellm with `Gemini API` to power autogen. This work is draws heavily on [autogen examples](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_custom_model.ipynb).

<h2 align=center> LiteLLM </h2>

We will need the `litellm` python client. This makes a call to gemini and returns a familiar OpenAI-like response that autogen can actually use. This greatly eases our design. It would be possible to complete this task without it but would require us to define this interface ourselves.

### Prerequisites
We will follow the litellm docs on [gemini](https://docs.litellm.ai/docs/providers/gemini).

Ensure you have the python client installed

```bash
pip install litellm
```

You may require a `GEMINI_API_KEY` environment variable set.

In [ ]:
from litellm import completion

<h2 align=center> Custom Autogen Model Class </h2>

This is an attempt to leverage LLM to ease the process of creating a custom model class for `autogen`. It needs to implement a particular protocol, the `ModelClientResponseProtocol.Choice.Message` protocol, to work.

In [ ]:
from typing import Protocol, Optional, List, Union, Dict

In [ ]:
# PROTOCOL CLASS
# Defines an OpenAI response interface
class ModelClientResponseProtocol(Protocol):
    class Choice(Protocol):
        class Message(Protocol):
            content: Optional[str]

        message: Message

    choices: List[Choice]
    model: str

In [69]:
# MODEL CLASS
class CustomModelClient:
    """
    A client class must implement the following methods:
    - create must return a response object that implements the ModelClientResponseProtocol
    - cost must return the cost of the response
    - get_usage must return a dict with the following keys:
        - prompt_tokens
        - completion_tokens
        - total_tokens
        - cost
        - model

    This class is used to create a client that can be used by OpenAIWrapper.
    The response returned from create must adhere to the ModelClientResponseProtocol but can be extended however needed.
    The message_retrieval method must be implemented to return a list of str or a list of messages from the response.
    """
    def __init__(self, config) -> None:
        self.model = config.get("model", "gemini/gemini-pro") 

    def create(self, params) -> ModelClientResponseProtocol:
        ## Get messages from agents
                
        response = completion(
            model=self.model, 
            messages=params["messages"]
        )
        
        return response

    def message_retrieval(
        self, response: ModelClientResponseProtocol
    ) -> Union[List[str], List[ModelClientResponseProtocol.Choice.Message]]:
        """
        Retrieve and return a list of strings or a list of Choice.Message from the response.

        NOTE: if a list of Choice.Message is returned, it currently needs to contain the fields of OpenAI's ChatCompletion Message object,
        since that is expected for function or tool calling in the rest of the codebase at the moment, unless a custom agent is being used.
        """
        choices = response.choices
        return [choice.message.content for choice in choices]

    def cost(self, response: ModelClientResponseProtocol) -> float:
        return 0

    @staticmethod
    def get_usage(response: ModelClientResponseProtocol) -> Dict:
        """Return usage summary of the response using RESPONSE_USAGE_KEYS."""
        return dict(response.usage)

In [70]:
config_list_custom = [
  {
      "model": "gemini/gemini-pro",             # May be overriden by the default gemini/gemini-pro
      "model_client_cls": "CustomModelClient",
  }
]

llm_config = {'config_list': config_list_custom}

<h2 align=center> Autogen Implementation </h2>

In [71]:
from autogen import AssistantAgent, UserProxyAgent

## Construct Agents

Consturct a simple conversation between a User proxy and an Assistent agent

In [72]:
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    },
)

[autogen.oai.client: 04-13 15:07:15] {418} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.


## Register the custom client class to the assistant agent
On specifying the custom config list, autogen informs us that:  
> [autogen.oai.client: 04-13 07:22:41] {419} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.


In [73]:
assistant.register_model_client(model_client_cls=CustomModelClient)

In [74]:
user_proxy.initiate_chat(assistant, message="Write python code to print Hello World!")

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):

Write python code to print Hello World!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


assistant (to user_proxy):

```python
print("Hello World!")
```

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


ChatResult(chat_id=None, chat_history=[{'content': 'Write python code to print Hello World!', 'role': 'assistant'}, {'content': '```python\nprint("Hello World!")\n```', 'role': 'user'}], summary='```python\nprint("Hello World!")\n```', cost=({'total_cost': 0}, {'total_cost': 0}), human_input=['exit'])